In [ ]:
#!pip install -r requirements.txt

In [ ]:
from azure.identity import AzureCliCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import AzureAISearchTool
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

# CONNECTION_STRING = "<HostName>; <AzureSubscriptionId>; <ResourceGroup>; <HubName>"
CONNECTION_STRING=os.getenv("AZURE_CONNECTION_STRING")
SEARCH_CONNECTION_NAME = os.getenv("AI_SEARCH_CONNECTION_NAME") # Your search connection name
SEARCH_INDEX_NAME = os.getenv("AI_SEARCH_INDEX_NAME") # Your search index name

client = AIProjectClient.from_connection_string(
credential=AzureCliCredential(),
conn_str=CONNECTION_STRING, # type: ignore
)

conn_id = next(c.id for c in client.connections.list() if c.name == SEARCH_CONNECTION_NAME)

search_tool = AzureAISearchTool(
    index_connection_id=conn_id,
    index_name=SEARCH_INDEX_NAME  # type: ignore
)

agent = client.agents.create_agent(
    model="gpt-4o-mini", # Model Deployment Name
    name="my-assistant",
    instructions="You are an Legal AI Expert. Use the Azure AI Search tool to search for legal documentations of the australian legal corpus.",
    tools=search_tool.definitions,
    tool_resources=search_tool.resources,
)

thread = client.agents.create_thread()

client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="If a Clare Valley winemaker inadvertently underpays their contributions to the Fund, what are the potential consequences, and is there a grace period for making up the shortfall??",
)

client.agents.create_and_process_run(
    thread_id=thread.id,
    agent_id=agent.id,
) # type: ignore

messages = client.agents.list_messages(thread_id=thread.id)
for message in messages['data']:
    role = message['role'].upper()
    content = ""
    for content_item in message.get("content", []):
        content = content_item.get("text", {}).get("value", "")
    print(f"{role}: {content}")

DEBUG:azure.identity._credentials.azure_cli:Executing subprocess with the following arguments ['C:\\Program Files (x86)\\Microsoft SDKs\\Azure\\CLI2\\wbin\\az.CMD', 'account', 'get-access-token', '--output', 'json', '--resource', 'https://management.azure.com']
INFO:azure.identity._internal.decorators:AzureCliCredential.get_token_info succeeded
DEBUG:azure.identity._internal.decorators:[Authenticated account] Client ID: 04b07795-8ddb-461a-bbee-02f9e1bf7b46. Tenant ID: 72f988bf-86f1-41af-91ab-2d7cd011db47. User Principal Name: fsunavala@microsoft.com. Object ID (user): c7b9be82-5bba-4f43-b49e-53051c7debcd
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://management.azure.com/subscriptions/ee787b9b-a25f-4d20-86e9-45fcea5253dd/resourceGroups/fsunavala-sandbox/providers/Microsoft.MachineLearningServices/workspaces/fsunavala-aml-sandbox/connections?api-version=REDACTED&includeAll=REDACTED'
Request method: 'GET'
Request headers:
    'Accept': 'application/json'
    

ASSISTANT: I was unable to find specific legal documentation detailing the consequences for a Clare Valley winemaker who inadvertently underpays their contributions to the fund, as well as any grace periods for correcting this shortfall. For accurate and detailed information, I recommend consulting legal resources related to the Clare Valley wine industry or the specific regulations governing the relevant fund. 

You may also want to contact a legal professional specializing in Australian agricultural or winemaking law to get precise guidance on this matter. Please let me know if you have other questions or need assistance with a different topic.
ASSISTANT: To determine the potential consequences for a Clare Valley winemaker who inadvertently underpays their contributions to the Fund, as well as whether there is a grace period for making up the shortfall, I will need to check the legal documentation for specific provisions. 

Please hold on while I conduct a more detailed search for th

## OpenAI Agents SDK Example 

In [ ]:
# Azure AI Search + OpenAI Agents SDK Integration
# pip install azure-search-documents openai "openai-agents>=0.3" python-dotenv

import os
from typing import List, Optional
from pydantic import BaseModel
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from openai import AsyncAzureOpenAI
from agents import (
    Agent, function_tool, Runner, OpenAIChatCompletionsModel,
    set_default_openai_client, set_tracing_disabled, ModelSettings
)

# Setup for notebooks
import nest_asyncio
nest_asyncio.apply()
load_dotenv()

# Initialize clients
search_client = SearchClient(
    os.environ.get("AZURE_SEARCH_SERVICE_ENDPOINT"),
    "your index-name",
    AzureKeyCredential(os.environ.get("AZURE_SEARCH_ADMIN_KEY"))
)

openai_client = AsyncAzureOpenAI(
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-10-21"),
    azure_deployment=os.environ.get("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")
)

# Configure SDK
set_default_openai_client(openai_client)
set_tracing_disabled(True)

# Define search tool
class SearchParams(BaseModel):
    query: str
    top: Optional[int] = 5

@function_tool
def search_azure_docs(params: SearchParams) -> List[dict]:
    """Search Azure AI Search documentation."""
    vector_query = VectorizableTextQuery(
        text=params.query,
        k_nearest_neighbors=params.top,
        fields="text_vector"
    )
    
    results = search_client.search(
        search_text=params.query,
        vector_queries=[vector_query],
        select=["chunk_id", "chunk", "title"],
        top=params.top
    )
    
    return [{
        "content": r.get("chunk", ""),
        "title": r.get("title", ""),
        "id": r.get("chunk_id", "")
    } for r in results]

# Create agent
agent = Agent(
    name="Azure AI Search Expert",
    instructions="""You are an Azure AI Search expert. Use 'search_azure_docs' tool 
    to find documentation.""",
    tools=[search_azure_docs],
    model=OpenAIChatCompletionsModel(
        model=os.environ.get("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME"),
        openai_client=openai_client
    ),
    model_settings=ModelSettings(temperature=0.3)
)

# Run agent
query = "How can I use vector search in Azure AI Search?"
result = Runner.run_sync(agent, query)
print(f"\nResponse:\n{result.final_output}")


Response:
To use vector search in Azure AI Search, you can follow these key points:

1. **Vector Storage**: Azure AI Search supports vector storage at the field level, allowing you to combine vector and non-vector fields in the same search index. You can create a vector store using the Create Index REST API or Azure SDK methods.

2. **Schema Design**: Design your schema based on your use case and intended vector retrieval pattern. Ensure to estimate index size and check service capacity.

3. **Hybrid Search**: Azure AI Search allows for hybrid scenarios where vector and keyword searches run in parallel, returning a unified result set. This often yields better results than using either method alone.

4. **Querying**: To perform a vector search, the query itself must be a vector. You can convert user input into a vector using an embedding library or API. Azure AI Search supports querying multiple vector fields and setting vector weights.

5. **Integration**: Vector search is available t

In [ ]:
# Step 1: Connect to Azure AI Foundry Inference & Azure AI Search
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import UserMessage

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

from azure.search.documents.models import VectorizableTextQuery

chat_client = ChatCompletionsClient(
    # endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"], # Ph4-mini serverless endpoint
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"], # Ph4-mini serverless endpoint
    credential=AzureKeyCredential(""),
    # credential=AzureKeyCredential(os.environ["AZURE_INFERENCE_CREDENTIAL"]),
)

search_client = SearchClient(
    endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    index_name=os.environ["AZURE_SEARCH_INDEX"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_KEY"])
)

# Step 2: Retrieve relevant documents from Azure AI Search
def retrieve_documents(query: str):
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="text_vector") # Assuming top=3
    results = search_client.search(search_text=query, vector_queries=[vector_query])
    return [doc["content"] for doc in results]

# Step 3: Generate answer using RAG!
def generate_rag_response(query: str):
    docs = retrieve_documents(query)
    context = "\n--\n".join(docs)
    prompt = f"""You are a helpful assistant. Use only the following context to answer the question. If the answer isn't in the context, say 'I don't know'.

    Context: {context}
    Question: {query}
    Answer:"""
    response = chat_client.complete(messages=[UserMessage(content=prompt)])
    return response.choices[0].message.content

# Example usage
user_query = "What is the capital of France?"
answer = generate_rag_response(user_query)
print(f"Q: {user_query}\nA: {answer}")

In [ ]:
# ------------------------------------
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
# ------------------------------------

"""
DESCRIPTION:
    This sample demonstrates how to use Azure AI Agent Service with function calling
    to search across two different Azure AI Search indexes: "fifa-legal-handbook" and 
    "trump-executive-orders" using hybrid search (text + vector) with built-in vectorizer.

USAGE:
    python hybrid_search_agent.py

    Before running the sample:

    pip install azure-ai-projects azure-identity azure-search-documents pydantic
    
    Set these environment variables with your own values:
    1) AZURE_CONNECTION_STRING - The project connection string
    2) AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME - The deployment name of the AI model (e.g., gpt-4o)
    3) AZURE_SEARCH_SERVICE_ENDPOINT - The endpoint URL of your Azure AI Search service
    4) AZURE_SEARCH_ADMIN_KEY - The API key for your Azure AI Search service
"""

import os
import time
import json
from enum import Enum
from typing import List, Optional, Dict, Any

from pydantic import BaseModel, Field

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import (
    FunctionTool, 
    RequiredFunctionToolCall, 
    SubmitToolOutputsAction, 
    ToolOutput,
    RunStatus
)
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential

# Initialize environment variables
PROJECT_CONNECTION_STRING = os.environ.get("AZURE_CONNECTION_STRING")
MODEL_DEPLOYMENT_NAME = os.environ.get("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")
SEARCH_ENDPOINT = os.environ.get("AZURE_SEARCH_SERVICE_ENDPOINT")
SEARCH_API_KEY = os.environ.get("AZURE_SEARCH_ADMIN_KEY")

# Initialize Azure AI Project client
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), 
    conn_str=PROJECT_CONNECTION_STRING
)

# Initialize search clients for both indexes
fifa_search_client = SearchClient(
    endpoint=SEARCH_ENDPOINT,
    index_name="fifa-legal-handbook",
    credential=AzureKeyCredential(SEARCH_API_KEY)
)

trump_search_client = SearchClient(
    endpoint=SEARCH_ENDPOINT,
    index_name="trump-executive-orders",
    credential=AzureKeyCredential(SEARCH_API_KEY)
)

# Define Pydantic models for search results
class DocumentType(str, Enum):
    FIFA_HANDBOOK = "fifa-handbook"
    EXECUTIVE_ORDER = "executive-order"

# Search result model for the agent
class SearchResult(BaseModel):
    chunk_id: str = Field(..., description="The unique identifier of the chunk")
    parent_id: str = Field(..., description="Parent document identifier")
    title: str = Field(..., description="The title of the document")
    chunk: str = Field(..., description="Content of the document chunk")
    document_type: DocumentType = Field(..., description="The type of document (fifa-handbook or executive-order)")
    relevance_score: float = Field(..., description="The relevance score of the search result", ge=0.0, le=1.0)
    metadata: Dict[str, Any] = Field(default_factory=dict, description="Additional metadata about the document")

class SearchResponse(BaseModel):
    query: str = Field(..., description="The search query used")
    results: List[SearchResult] = Field(..., description="List of search results")
    total_count: int = Field(..., description="Total number of matching documents")
    index_name: str = Field(..., description="Name of the index searched")

# Define search functions with hybrid search
def search_fifa_legal_handbook(query: str, top: int = 3, filter: str = None) -> str:
    """
    Search the FIFA legal handbook using hybrid search (text + vector).
    
    Args:
        query: The search query text
        top: Maximum number of results to return (default: 3)
        filter: Optional OData filter expression
        
    Returns:
        JSON string of search results
    """
    try:
        print(f"Searching FIFA legal handbook for: {query}")
        
        # Set up search parameters
        search_params = {
            'top': top,
            'include_total_count': True,
            'select': ["chunk_id", "parent_id", "title", "chunk"]
        }
        
        # Add filter if provided
        if filter:
            search_params['filter'] = filter
            
        # Use vectorizable text query for vector search with built-in vectorizer
        vector_query = VectorizableTextQuery(
            text=query,
            k_nearest_neighbors=top,
            fields="text_vector"
        )
        search_params['vector_queries'] = [vector_query]
            
        # Perform hybrid search
        search_results = fifa_search_client.search(
            search_text=query,
            **search_params
        )
        
        # Process results
        results = []
        for result in search_results:
            results.append(SearchResult(
                chunk_id=result.get("chunk_id", "unknown"),
                parent_id=result.get("parent_id", "unknown"),
                title=result.get("title", "FIFA Handbook Section"),
                chunk=result.get("chunk", ""),
                document_type=DocumentType.FIFA_HANDBOOK,
                relevance_score=result.get("@search.score", 0.0),
                metadata={
                    "index": "fifa-legal-handbook"
                }
            ))
        
        # Create response model
        response = SearchResponse(
            query=query,
            results=results,
            total_count=search_results.get_count() or len(results),
            index_name="fifa-legal-handbook"
        )
        
        return response.model_dump_json()
    
    except Exception as e:
        error_response = {
            "error": str(e),
            "query": query,
            "index_name": "fifa-legal-handbook"
        }
        print(f"Error searching FIFA legal handbook: {e}")
        return json.dumps(error_response)

def search_trump_executive_orders(query: str, top: int = 3, filter: str = None) -> str:
    """
    Search Trump executive orders using hybrid search (text + vector).
    
    Args:
        query: The search query text
        top: Maximum number of results to return (default: 3)
        filter: Optional OData filter expression
        
    Returns:
        JSON string of search results
    """
    try:
        print(f"Searching Trump executive orders for: {query}")
        
        # Set up search parameters
        search_params = {
            'top': top,
            'include_total_count': True,
            'select': ["chunk_id", "parent_id", "title", "chunk"]
        }
        
        # Add filter if provided
        if filter:
            search_params['filter'] = filter
            
        # Use vectorizable text query for vector search with built-in vectorizer
        vector_query = VectorizableTextQuery(
            text=query,
            k_nearest_neighbors=top,
            fields="text_vector"
        )
        search_params['vector_queries'] = [vector_query]
            
        # Perform hybrid search
        search_results = trump_search_client.search(
            search_text=query,
            **search_params
        )
        
        # Process results
        results = []
        for result in search_results:
            results.append(SearchResult(
                chunk_id=result.get("chunk_id", "unknown"),
                parent_id=result.get("parent_id", "unknown"),
                title=result.get("title", "Executive Order"),
                chunk=result.get("chunk", ""),
                document_type=DocumentType.EXECUTIVE_ORDER,
                relevance_score=result.get("@search.score", 0.0),
                metadata={
                    "index": "trump-executive-orders"
                }
            ))
        
        # Create response model
        response = SearchResponse(
            query=query,
            results=results,
            total_count=search_results.get_count() or len(results),
            index_name="trump-executive-orders"
        )
        
        return response.model_dump_json()
    
    except Exception as e:
        error_response = {
            "error": str(e),
            "query": query,
            "index_name": "trump-executive-orders"
        }
        print(f"Error searching Trump executive orders: {e}")
        return json.dumps(error_response)

# Create function tool with search functions
search_functions = {
    search_fifa_legal_handbook,
    search_trump_executive_orders
}

function_tool = FunctionTool(functions=search_functions)

def main():
    """Main function to run the agent search example"""
    
    with project_client:
        # Create an agent with the search functions
        print("Creating agent...")
        agent = project_client.agents.create_agent(
            model=MODEL_DEPLOYMENT_NAME,
            name="fifa-trump-research-agent",
            instructions="""You are a research assistant specializing in international sports governance and U.S. policy. 
            
Your tasks include:
1. Analyze how Trump executive orders may have impacted or facilitated FIFA World Cup preparations in North America
2. Identify connections between U.S. executive policies and FIFA regulations for the 2026 World Cup
3. Research legal frameworks established by executive orders that relate to international sporting events
4. Explore how executive actions might address FIFA's requirements for host nations

Use the search functions to find relevant information:
- search_fifa_legal_handbook: Search FIFA regulations and requirements
- search_trump_executive_orders: Search executive orders related to your queries

When searching, try different search terms to explore various aspects of the relationship between executive orders and FIFA regulations. For example:
- Search for "visa", "border", "security", "infrastructure", "commercial rights" in both indexes
- Use specific terms from FIFA regulations when searching executive orders and vice versa

Provide comprehensive analysis of how executive orders and FIFA regulations intersect, with specific examples backed by your search results. Always cite your sources by referencing document IDs.
            """,
            tools=function_tool.definitions,
        )
        print(f"Created agent, ID: {agent.id}")

        # Create a thread
        thread = project_client.agents.create_thread()
        print(f"Created thread, ID: {thread.id}")

        # Create a user message
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content="I'm researching how Trump executive orders might have facilitated FIFA World Cup 2026 implementation in North America. Can you find connections between any executive orders and FIFA's requirements for host countries?",
        )
        print(f"Created message, ID: {message.id}")

        # Create a run
        run = project_client.agents.create_run(thread_id=thread.id, agent_id=agent.id)
        print(f"Created run, ID: {run.id}")

        # Process the run and handle function calls
        while run.status in ["queued", "in_progress", "requires_action"]:
            time.sleep(1)
            run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

            if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
                tool_calls = run.required_action.submit_tool_outputs.tool_calls
                if not tool_calls:
                    print("No tool calls provided - cancelling run")
                    project_client.agents.cancel_run(thread_id=thread.id, run_id=run.id)
                    break

                tool_outputs = []
                for tool_call in tool_calls:
                    if isinstance(tool_call, RequiredFunctionToolCall):
                        try:
                            print(f"Executing tool call: {tool_call.function.name} with args: {tool_call.function.arguments}")
                            output = function_tool.execute(tool_call)
                            tool_outputs.append(
                                ToolOutput(
                                    tool_call_id=tool_call.id,
                                    output=output,
                                )
                            )
                        except Exception as e:
                            print(f"Error executing tool_call {tool_call.id}: {e}")
                            # Provide a fallback response instead of failing completely
                            tool_outputs.append(
                                ToolOutput(
                                    tool_call_id=tool_call.id,
                                    output=json.dumps({
                                        "error": str(e),
                                        "query": json.loads(tool_call.function.arguments).get("query", ""),
                                        "results": []
                                    })
                                )
                            )

                print(f"Submitting {len(tool_outputs)} tool outputs")
                if tool_outputs:
                    project_client.agents.submit_tool_outputs_to_run(
                        thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
                    )

            print(f"Current run status: {run.status}")

        print(f"Run completed with status: {run.status}")

        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        print("\n=== CONVERSATION ===")
        for msg in messages.data:
            role = msg.role.upper()
            content = ""
            for content_item in msg.content:
                if hasattr(content_item, "text"):
                    content = content_item.text.value
            print(f"\n{role}: {content}")

        # Clean up
        project_client.agents.delete_agent(agent.id)
        print("\nDeleted agent")

if __name__ == "__main__":
    main()

Creating agent...
Created agent, ID: asst_qIxMcxOeovU38YjlOVZFBm8e
Created thread, ID: thread_mFgyrqtPvngJ8CFEE3He8A8I
Created message, ID: msg_PbCP6RdmvwlHE7vOtpA23L0L
Created run, ID: run_Rr6wUpZ1teXdUWRbcezSKu9s
Current run status: RunStatus.IN_PROGRESS
Executing tool call: mock_search_trump_executive_orders
MOCK: Searching Trump executive orders for: FIFA World Cup
Executing tool call: mock_search_fifa_legal_handbook
MOCK: Searching FIFA legal handbook for: host country requirements
Submitting 2 tool outputs
Current run status: RunStatus.REQUIRES_ACTION
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.COMPLETED
Run completed with status: RunStatus.COMPLETED

=== CONVERSATION ===

ASSISTANT: The intersection of Trump executive orders and FIFA requirements for the 2026 World Cup reveals several c

In [ ]:
# ------------------------------------
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
# ------------------------------------

"""
DESCRIPTION:
    This sample demonstrates how to use Azure AI Agent Service with function calling
    to search across two different Azure AI Search indexes: "fifa-legal-handbook" and 
    "trump-executive-orders" using hybrid search with built-in vectorizer.
    Includes detailed run step visualization for better debugging.

USAGE:
    python enhanced_search_agent.py

    Before running the sample:

    pip install azure-ai-projects azure-identity azure-search-documents pydantic
    
    Set these environment variables with your own values:
    1) AZURE_CONNECTION_STRING - The project connection string
    2) AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME - The deployment name of the AI model (e.g., gpt-4o)
    3) AZURE_SEARCH_SERVICE_ENDPOINT - The endpoint URL of your Azure AI Search service
    4) AZURE_SEARCH_ADMIN_KEY - The API key for your Azure AI Search service
"""

import os
import time
import json
from enum import Enum
from typing import List, Optional, Dict, Any

from pydantic import BaseModel, Field

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import (
    FunctionTool, 
    RequiredFunctionToolCall, 
    SubmitToolOutputsAction, 
    ToolOutput,
    RunStatus
)
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential

# Initialize environment variables
PROJECT_CONNECTION_STRING = os.environ.get("AZURE_CONNECTION_STRING")
MODEL_DEPLOYMENT_NAME = os.environ.get("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")
SEARCH_ENDPOINT = os.environ.get("AZURE_SEARCH_SERVICE_ENDPOINT")
SEARCH_API_KEY = os.environ.get("AZURE_SEARCH_ADMIN_KEY")

# Validate required environment variables
if not all([PROJECT_CONNECTION_STRING, MODEL_DEPLOYMENT_NAME, SEARCH_ENDPOINT, SEARCH_API_KEY]):
    raise ValueError("Required environment variables are missing. Please set AZURE_CONNECTION_STRING, AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME, AZURE_SEARCH_SERVICE_ENDPOINT, and AZURE_SEARCH_ADMIN_KEY.")

# Initialize Azure AI Project client
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), 
    conn_str=PROJECT_CONNECTION_STRING
)

# Initialize search clients for both indexes
fifa_search_client = SearchClient(
    endpoint=SEARCH_ENDPOINT,
    index_name="fifa-legal-handbook",
    credential=AzureKeyCredential(SEARCH_API_KEY)
)

trump_search_client = SearchClient(
    endpoint=SEARCH_ENDPOINT,
    index_name="trump-executive-orders",
    credential=AzureKeyCredential(SEARCH_API_KEY)
)

# Define Pydantic models for search results
class DocumentType(str, Enum):
    FIFA_HANDBOOK = "fifa-handbook"
    EXECUTIVE_ORDER = "executive-order"

# Search result model for the agent
class SearchResult(BaseModel):
    chunk_id: str = Field(..., description="The unique identifier of the chunk")
    parent_id: str = Field(..., description="Parent document identifier")
    title: str = Field(..., description="The title of the document")
    chunk: str = Field(..., description="Content of the document chunk")
    document_type: DocumentType = Field(..., description="The type of document (fifa-handbook or executive-order)")
    relevance_score: float = Field(..., description="The relevance score of the search result", ge=0.0, le=1.0)
    metadata: Dict[str, Any] = Field(default_factory=dict, description="Additional metadata about the document")

class SearchResponse(BaseModel):
    query: str = Field(..., description="The search query used")
    results: List[SearchResult] = Field(..., description="List of search results")
    total_count: int = Field(..., description="Total number of matching documents")
    index_name: str = Field(..., description="Name of the index searched")

# Define search functions with hybrid search
def search_fifa_legal_handbook(query: str, top: int = 3, filter: str = None) -> str:
    """
    Search the FIFA legal handbook using hybrid search (text + vector).
    
    Args:
        query: The search query text
        top: Maximum number of results to return (default: 3)
        filter: Optional OData filter expression
        
    Returns:
        JSON string of search results
    """
    try:
        print(f"Searching FIFA legal handbook for: {query}")
        
        # Set up search parameters
        search_params = {
            'top': top,
            'include_total_count': True,
            'select': ["chunk_id", "parent_id", "title", "chunk"]
        }
        
        # Add filter if provided
        if filter:
            search_params['filter'] = filter
            
        # Use vectorizable text query for vector search with built-in vectorizer
        vector_query = VectorizableTextQuery(
            text=query,
            k_nearest_neighbors=top,
            fields="text_vector"
        )
        search_params['vector_queries'] = [vector_query]
            
        # Perform hybrid search
        search_results = fifa_search_client.search(
            search_text=query,  # Include text for hybrid search
            **search_params
        )
        
        # Process results
        results = []
        for result in search_results:
            results.append(SearchResult(
                chunk_id=result.get("chunk_id", "unknown"),
                parent_id=result.get("parent_id", "unknown"),
                title=result.get("title", "FIFA Handbook Section"),
                chunk=result.get("chunk", ""),
                document_type=DocumentType.FIFA_HANDBOOK,
                relevance_score=result.get("@search.score", 0.0),
                metadata={
                    "index": "fifa-legal-handbook"
                }
            ))
        
        # Create response model
        response = SearchResponse(
            query=query,
            results=results,
            total_count=search_results.get_count() or len(results),
            index_name="fifa-legal-handbook"
        )
        
        return response.model_dump_json()
    
    except Exception as e:
        error_response = {
            "error": str(e),
            "query": query,
            "index_name": "fifa-legal-handbook"
        }
        print(f"Error searching FIFA legal handbook: {e}")
        return json.dumps(error_response)

def search_trump_executive_orders(query: str, top: int = 3, filter: str = None) -> str:
    """
    Search Trump executive orders using hybrid search (text + vector).
    
    Args:
        query: The search query text
        top: Maximum number of results to return (default: 3)
        filter: Optional OData filter expression
        
    Returns:
        JSON string of search results
    """
    try:
        print(f"Searching Trump executive orders for: {query}")
        
        # Set up search parameters
        search_params = {
            'top': top,
            'include_total_count': True,
            'select': ["chunk_id", "parent_id", "title", "chunk"]
        }
        
        # Add filter if provided
        if filter:
            search_params['filter'] = filter
            
        # Use vectorizable text query for vector search with built-in vectorizer
        vector_query = VectorizableTextQuery(
            text=query,
            k_nearest_neighbors=top,
            fields="text_vector"
        )
        search_params['vector_queries'] = [vector_query]
            
        # Perform hybrid search
        search_results = trump_search_client.search(
            search_text=query,  # Include text for hybrid search
            **search_params
        )
        
        # Process results
        results = []
        for result in search_results:
            results.append(SearchResult(
                chunk_id=result.get("chunk_id", "unknown"),
                parent_id=result.get("parent_id", "unknown"),
                title=result.get("title", "Executive Order"),
                chunk=result.get("chunk", ""),
                document_type=DocumentType.EXECUTIVE_ORDER,
                relevance_score=result.get("@search.score", 0.0),
                metadata={
                    "index": "trump-executive-orders"
                }
            ))
        
        # Create response model
        response = SearchResponse(
            query=query,
            results=results,
            total_count=search_results.get_count() or len(results),
            index_name="trump-executive-orders"
        )
        
        return response.model_dump_json()
    
    except Exception as e:
        error_response = {
            "error": str(e),
            "query": query,
            "index_name": "trump-executive-orders"
        }
        print(f"Error searching Trump executive orders: {e}")
        return json.dumps(error_response)

# Create function tool with search functions
search_functions = {
    search_fifa_legal_handbook,
    search_trump_executive_orders
}

function_tool = FunctionTool(functions=search_functions)

def visualize_run_steps(thread_id, run_id):
    """Fetch and visualize the detailed run steps for better debugging"""
    print("\n=== DETAILED RUN STEPS ===")
    
    # Fetch run steps
    run_steps = project_client.agents.list_run_steps(thread_id=thread_id, run_id=run_id)
    
    for step in run_steps.data:
        print(f"\nStep ID: {step.id}")
        print(f"Step Type: {step.type}")
        print(f"Step Status: {step.status}")
        
        # Check for tool calls
        step_details = step.step_details
        if hasattr(step_details, "tool_calls") and step_details.tool_calls:
            print("\n  Tool Calls:")
            for call in step_details.tool_calls:
                print(f"    Tool Call ID: {call.id}")
                print(f"    Tool Call Type: {call.type}")
                
                # Handle function tool calls
                if call.type == "function" and hasattr(call, "function"):
                    print(f"    Function Name: {call.function.name}")
                    print(f"    Function Arguments: {call.function.arguments}")
                    
                    # If this step has outputs, show them
                    if hasattr(step_details, "tool_call_id") and step_details.tool_call_id == call.id and hasattr(step_details, "output"):
                        print("\n    Function Output:")
                        # Try to format JSON output for better readability
                        try:
                            output_json = json.loads(step_details.output)
                            formatted_json = json.dumps(output_json, indent=4)
                            print(f"    {formatted_json}")
                        except:
                            print(f"    {step_details.output}")
        
        # Check for submitted tool outputs
        if hasattr(step_details, "tool_outputs") and step_details.tool_outputs:
            print("\n  Tool Outputs Submitted:")
            for output in step_details.tool_outputs:
                print(f"    Tool Call ID: {output.tool_call_id}")
                # Try to format JSON output for better readability
                try:
                    output_json = json.loads(output.output)
                    formatted_json = json.dumps(output_json, indent=4)
                    print(f"    Output: {formatted_json}")
                except:
                    print(f"    Output: {output.output}")
                    
        print("-" * 50)  # Separator between steps

def main():
    """Main function to run the agent search example"""
    
    with project_client:
        # Create an agent with the search functions
        print("Creating agent...")
        agent = project_client.agents.create_agent(
            model=MODEL_DEPLOYMENT_NAME,
            name="fifa-trump-research-agent",
            instructions="""You are a research assistant specializing in international sports governance and U.S. policy. 
            
Your tasks include:
1. Analyze how Trump executive orders may have impacted or facilitated FIFA World Cup preparations in North America
2. Identify connections between U.S. executive policies and FIFA regulations for the 2026 World Cup
3. Research legal frameworks established by executive orders that relate to international sporting events
4. Explore how executive actions might address FIFA's requirements for host nations

Use the search functions to find relevant information:
- search_fifa_legal_handbook: Search FIFA regulations and requirements
- search_trump_executive_orders: Search executive orders related to your queries

When searching, try different search terms to explore various aspects of the relationship between executive orders and FIFA regulations. For example:
- Search for "visa", "border", "security", "infrastructure", "commercial rights" in both indexes
- Use specific terms from FIFA regulations when searching executive orders and vice versa

IMPORTANT GUIDELINES:
1. Always cite your sources by referencing document IDs (chunk_id and parent_id).
2. Include direct quotes from the documents when making claims about their content.
3. Be explicit when information is not found rather than speculating.
4. Make it clear when you are providing contextual information versus reporting facts from the indexes.
5. If you find no direct connections between executive orders and FIFA requirements, state this clearly.

Provide a comprehensive analysis of how executive orders and FIFA regulations may intersect, with specific examples backed by your search results.
            """,
            tools=function_tool.definitions,
        )
        print(f"Created agent, ID: {agent.id}")

        # Create a thread
        thread = project_client.agents.create_thread()
        print(f"Created thread, ID: {thread.id}")

        # Create a user message
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content="I'm researching how Trump executive orders as of 2025 in his new term might have facilitated FIFA World Cup 2026 implementation in North America. Can you find connections between any executive orders and FIFA's requirements for host countries?",
        )
        print(f"Created message, ID: {message.id}")

        # Create a run
        run = project_client.agents.create_run(thread_id=thread.id, agent_id=agent.id)
        print(f"Created run, ID: {run.id}")

        # Process the run and handle function calls
        while run.status in ["queued", "in_progress", "requires_action"]:
            time.sleep(1)
            run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

            if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
                tool_calls = run.required_action.submit_tool_outputs.tool_calls
                if not tool_calls:
                    print("No tool calls provided - cancelling run")
                    project_client.agents.cancel_run(thread_id=thread.id, run_id=run.id)
                    break

                tool_outputs = []
                for tool_call in tool_calls:
                    if isinstance(tool_call, RequiredFunctionToolCall):
                        try:
                            print(f"Executing tool call: {tool_call.function.name} with args: {tool_call.function.arguments}")
                            output = function_tool.execute(tool_call)
                            tool_outputs.append(
                                ToolOutput(
                                    tool_call_id=tool_call.id,
                                    output=output,
                                )
                            )
                        except Exception as e:
                            print(f"Error executing tool_call {tool_call.id}: {e}")
                            # Provide a fallback response instead of failing completely
                            tool_outputs.append(
                                ToolOutput(
                                    tool_call_id=tool_call.id,
                                    output=json.dumps({
                                        "error": str(e),
                                        "query": json.loads(tool_call.function.arguments).get("query", ""),
                                        "results": []
                                    })
                                )
                            )

                print(f"Submitting {len(tool_outputs)} tool outputs")
                if tool_outputs:
                    project_client.agents.submit_tool_outputs_to_run(
                        thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
                    )

            print(f"Current run status: {run.status}")

        print(f"Run completed with status: {run.status}")
        
        # Visualize the run steps for better debugging
        visualize_run_steps(thread.id, run.id)

        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        print("\n=== CONVERSATION ===")
        for msg in messages.data:
            role = msg.role.upper()
            content = ""
            for content_item in msg.content:
                if hasattr(content_item, "text"):
                    content = content_item.text.value
            print(f"\n{role}: {content}")

        # Clean up
        project_client.agents.delete_agent(agent.id)
        print("\nDeleted agent")

if __name__ == "__main__":
    main()

Creating agent...
Created agent, ID: asst_VanDDC56RdbuFsAvJU4rjeyc
Created thread, ID: thread_SFzZrrTrQxjFuo3OWEWdVvsQ
Created message, ID: msg_MnPfsEuQBfWEHvpMfmhWnD0K
Created run, ID: run_DyRiijTcuEL5zZ4EEqkmUjhX
Executing tool call: search_trump_executive_orders with args: {"query": "FIFA World Cup 2026", "top": 5}
Searching Trump executive orders for: FIFA World Cup 2026
Executing tool call: search_fifa_legal_handbook with args: {"query": "host country requirements", "top": 5}
Searching FIFA legal handbook for: host country requirements
Submitting 2 tool outputs
Current run status: RunStatus.REQUIRES_ACTION
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.COMPLETED
Run c